In [1]:
import torch
import os
import numpy as np
import pandas as pd
import sklearn as sk
from imblearn.over_sampling import RandomOverSampler
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn, optim
import torch.nn.functional as F
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
from torch.utils.data import Dataset,DataLoader
print('Package import Completed!')

def props_to_onehot(props):
    props=props.detach().numpy()
    a=np.argmax(props,axis=1)
    b=np.zeros((len(a),props.shape[1]))
    b[np.arange(len(a)),a]=1
    return b

Package import Completed!


In [3]:
filePath='OULA.DataSet\\'
df_stuAss=pd.read_csv(filePath + 'studentAssessment.csv')
df_stuAss['score'].fillna(0,inplace=True)
df_Ass=pd.read_csv(filePath + 'Assessments.csv')
df_Courses=pd.read_csv(filePath + 'courses.csv')
df_Stu=pd.read_csv(filePath + 'studentInfo.csv')
df_stuAss=pd.merge(df_stuAss,df_Ass,on='id_assessment')
df_stuAss=pd.merge(df_stuAss,df_Stu,on=['id_student','code_module','code_presentation'])
#df_stuAss.head()

In [10]:
test_percent=0.25
batch_size=200
learnrate=0.001
countEpoch=500

lossList=[];
res=[];
for index,course in df_Courses.iterrows():
    print(course['code_module'] + ',' + course['code_presentation'] + ' Start...')
    thisCourseLoss=[]
    thisCourseLoss.append(course['code_module']+'_'+course['code_presentation'])
    df_CurrentCourse=df_stuAss[(df_stuAss['code_module']==course['code_module']) & (df_stuAss['code_presentation']==course['code_presentation'])]
    
    df_CurrentCourse=df_CurrentCourse.loc[:,['id_student','date_submitted','score','final_result']]
    
    countStu=len(df_CurrentCourse['id_student'].unique())
    maxAssessCount=df_CurrentCourse.groupby('id_student').count()['score'].max()
    
    listCurrentStu=[]
    listFinalRes=[]
    finalRes='None'
    for id in df_CurrentCourse['id_student'].unique():
        df_id_ass=df_CurrentCourse[df_CurrentCourse['id_student']==id]
        df_id_ass=df_id_ass.sort_values(by='date_submitted',ascending=True)
        newlist = []
        record=[]
        for index,row in df_id_ass.iterrows():
            newlist.append(row['score'])
            finalRes=row['final_result']
        newlist += [0]*(maxAssessCount-len(newlist))
        listCurrentStu.append(newlist)
        listFinalRes.append(finalRes)  
    
    ls_x,ls_test_x,ls_y,ls_testy = train_test_split(listCurrentStu,listFinalRes,test_size=test_percent)
    train_input = torch.Tensor(ls_x)
    train_target=torch.from_numpy(np.array(pd.get_dummies(pd.Series(ls_y))))
    test_input = torch.Tensor(ls_test_x)
    test_target=torch.from_numpy(np.array(pd.get_dummies(pd.Series(ls_testy))))
    
    countFeatures=train_input.size()[1] 
    countTargets=train_target.size()[1] 
    
    class MData(Dataset):
        def __init__(self,x,y):
            self.data=x
            self._x=x
            self._y=y
        def __getitem__(self,index):
            return self._x[index].to(torch.float32),self._y[index].to(torch.float32)
        def __len__(self):
            return len(self.data)
        
    mydata=MData(train_input,train_target)
    dataloader=DataLoader(mydata,batch_size=batch_size,shuffle=False,drop_last=False,num_workers=0)

    class FNN(nn.Module):
        def __init__(self, input_dim):
            super(FNN, self).__init__()
            self.net = nn.Sequential(
                nn.Linear(input_dim, input_dim),
                nn.ReLU(),
                nn.Linear(input_dim, 512),
                nn.ReLU(),
                nn.Linear(512, 256),
                nn.ReLU(),
                nn.Linear(256, 128),
                nn.ReLU(),
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, countTargets),
                nn.Softmax(dim=1)
            )
            self.criterion = nn.CrossEntropyLoss()
        def forward(self, x):
            return self.net(x)
    
    model=FNN(countFeatures)
    optimizer = torch.optim.Adam(model.parameters(),lr=learnrate)
    for epoch in range(countEpoch):
        for i,data in enumerate(dataloader):
            x,y=data
            if (RNN):
                x=x.reshape(-1,countFeatures,1)
            yhat=model(x)
            loss=model.criterion(yhat,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        testloss=model.criterion(model(train_input),train_target.to(torch.float32))
        thisCourseLoss.append(testloss.item())

    model.eval()
    test_y=model(test_input)
    test_y=props_to_onehot(test_y)
    
    resThisCourse=[]
    print(course['code_module'] + ',' + course['code_presentation'] + ' Report Start...')
    resThisCourse.append(course['code_module'] + '_' + course['code_presentation'])
    resThisCourse.append(sk.metrics.accuracy_score(test_target,test_y))
    resThisCourse.append(sk.metrics.precision_score(test_target,test_y,average='macro'))
    resThisCourse.append(sk.metrics.recall_score(test_target,test_y,average='macro'))
    resThisCourse.append(sk.metrics.f1_score(test_target,test_y,average='macro'))
    print('Accuracy: ' + str(sk.metrics.accuracy_score(test_target,test_y)))
    res.append(resThisCourse)
    lossList.append(thisCourseLoss)
dfRes=pd.DataFrame(res)
dfRes.to_csv("OULA.Exp.Res\\Assessments.Res.csv",index=False)

AAA,2013J Start...
AAA,2013J Report Start...


C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.8152173913043478
AAA,2014J Start...
AAA,2014J Report Start...


C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.8
BBB,2013J Start...
BBB,2013J Report Start...


...

Accuracy: 0.7073170731707317
GGG,2014J Start...
GGG,2014J Report Start...


C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.6821192052980133
GGG,2014B Start...
GGG,2014B Report Start...


C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.6416184971098265
